# Texas Tow Trucks

We're going to scrape some [tow trucks in Texas](https://www.tdlr.texas.gov/tools_search/). 

**You can use ChatGPT for this one**, but I'd first try to do it on your own. Link to any conversations you have with ChatGPT. **Use the share button in the top right – not just the URL of your chat**.

# Part One: Building a company list

Search for businesses with the word **WRECK** in their names.

* **Tip:** Start by scraping the first page to a dataframe, then expand to a loop that combines all of the pages. Finally combine all of the dataframes with `pd.concat`. You might find [this page helpful](https://jonathansoma.com/everything/scraping/pagination/), although the "On an interactive site" example uses Selenium instead of Playwright. You should be able to figure out how to change it!
* **Tip:** You can't just do a `try`/`except`, because even if you ask for page 99999 it will always give you the last page again! Watch out that you don't get stuck in an infinite loop!
* **Tip:** Try to figure it out yourself before you submit to the gods of AI

In [27]:
# ## i have been trying for an hour, I'm tired ... 
# from playwright.async_api import async_playwright
# playwright = await async_playwright().start()
# browser = await playwright.chromium.launch(headless=False)
# page = await browser.new_page()
# await page.goto("https://www.tdlr.texas.gov/tools_search/mccs_search.asp")
# await page.fill("input[id='namedata']", "WRECK")
# await page.select_option('select[name="name_carrier_type"]', {'label': 'DBA Name'})
# #await page.click("button[name='searchtype']")
# await page.click("button[type='submit']")
# from bs4 import BeautifulSoup
# html = await page.content()
# from bs4 import BeautifulSoup
# table = soup_doc.find("div",id_="content")
# print(table)

In [51]:
from playwright.async_api import async_playwright
import io 
import pandas as pd

playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless=False)
page = await browser.new_page()
await page.goto("https://www.tdlr.texas.gov/tools_search/mccs_search.asp")
await page.click('input#namebutton')
await page.select_option('select#name_carrier_type', value="DBA")
await page.fill('input#namedata', 'WRECK')
await page.locator('button[name="proc"]').click()
await page.wait_for_load_state('networkidle')
html = await page.content()

tables = pd.read_html(io.StringIO(html))
print(tables)

[                                                   0  \
0  Tow Company and Vehicle Facility Storage Inqui...   

                               1  
0  Total number of records : 645  ,               0   1
0  Page 1 of 33 NaN,                                    0                                   1  \
0                           Customer                            DBA Name   
1                   271 VENTURES LLC                        271 WRECKERS   
2                   271 VENTURES LLC                        271 WRECKERS   
3            290 WRECKER SERVICE INC                     HWY 290 WRECKER   
4           290 WRECKER SERVICE INC.                     HWY 290 WRECKER   
5          290 WRECKER SERVICE, INC.             HWY 290 WRECKER SERVICE   
6          290 WRECKER SERVICE, INC.                     HWY 290 WRECKER   
7          290 WRECKER SERVICE, INC.                     HWY 290 WRECKER   
8              4A'S ENTERPRISES, INC             ERIKA'S WRECKER SERVICE   
9             

### Cleanup

If you haven't already, rename the columns to be:
    
    * Customer
    * DBA Name
    * TDLR Number
    * City
    * State
    * Zip code

and remove all of the rows where the customer name is `Customer`.

In [59]:
main_table = tables[2]

main_table.columns = ["Customer", "DBA Name", "TDLR Number", "City", "State", "Zip code"]

cleaned_table = main_table[main_table["Customer"] != "Customer"]

cleaned_table


,Customer,DBA Name,TDLR Number,City,State,Zip code
1,271 VENTURES LLC,271 WRECKERS,006428802C,GLADEWATER,TX,75647
2,271 VENTURES LLC,271 WRECKERS,0642881VSF,GLADEWATER,TX,75647
3,290 WRECKER SERVICE INC,HWY 290 WRECKER,0656637VSF,HOCKLEY,TX,77447
4,290 WRECKER SERVICE INC.,HWY 290 WRECKER,0650616VSF,HOCKLEY,TX,77447
5,"290 WRECKER SERVICE, INC.",HWY 290 WRECKER SERVICE,0657944VSF (Expired!),HOCKLEY,TX,77447
6,"290 WRECKER SERVICE, INC.",HWY 290 WRECKER,0545753VSF,HOCKLEY,TX,77447
7,"290 WRECKER SERVICE, INC.",HWY 290 WRECKER,005697854C,HOCKLEY,TX,77484
8,"4A'S ENTERPRISES, INC",ERIKA'S WRECKER SERVICE,0606250VSF,CORPUS CHRISTI,TX,78427
9,"4A'S ENTERPRISES, INC.",ERIKA'S WRECKER SERVICE,006061621C,CORPUS CHRISTI,TX,78247
10,A & L DELGADO'S CORPORATION,DELGADO'S WRECKER SERVICE LOT NO 2,0643569VSF,EAGLE LAKE,TX,77434


## Save as `wreckers.csv`

In [64]:
import pandas as pd 
df = pd.DataFrame(cleaned_table)
output_file = 'tow_output.csv'
df.to_csv(output_file, index=False)

# Part Two: Company info

> You can use whatever tool you'd like for this, but form submission doesn't necessarily mean Playwright! If you want to go the `requests` route instead, feel free to [look at this page](https://jonathansoma.com/everything/scraping/pretending-to-be-a-browser/) about several ways to pretend to be a browser.

## Step 1: Scraping one page

Try searching from the [tools page](https://www.tdlr.texas.gov/tools_search/) for the TDLR Number `006556161C`. From the results page, scrape the:

* Business name
* Phone number
* License status
* Physical address

And save the results into a dictionary. Also include `TDLR Number` in the dictionary. **Print the dictionary**.

> ***Tip:** It's best if each item has its own key, but **it's fine to pull "larger" sections of the page and split them up in pandas later on***
>
> ***Tip:** Be sure you don't forget to include `TDLR Number` in the dictionary!*

In [65]:
page = await browser.new_page()
await page.goto("https://www.tdlr.texas.gov/tools_search/mccs_search.asp")
await page.click('input#mcrbutton')
await page.fill('input#mcrdata', '006556161C')
await page.locator('button[name="proc"]').click()
await page.wait_for_load_state('networkidle')
html = await page.content()

tables = pd.read_html(io.StringIO(html))
main_table = tables[0]
#print(tables)

[                                                   0  \
0  MCCS_Seach Functions  Company Information  IMP...   
1                                Company Information   
2  IMPORTANT: A tow company is not authorized to ...   
3                               Company Information:   
4      Name: DIRTY SOUTH TRANSPORT AND RECOVERY, LLC   
5     Owner/Officer: ANGELA RACHELL RAINES / MANAGER   
6        Owner/Officer: MATTHEW W RAINES / PRESIDENT   
7                                Phone: 713-259-5445   
8                           Certificate Information:   
9                       Number: 006556161C (Expired)   

                                                   1  
0                                                NaN  
1                                                NaN  
2                                                NaN  
3                                                NaN  
4                                       DBA: NO DATA  
5                                                NaN

In [67]:
from bs4 import BeautifulSoup
soup_doc = BeautifulSoup(html, "html.parser")
business_name = soup_doc.find(

## Step 2: Move into one cell

Move the code above all into one cell that relies on the variable `tdlr_number`.

Add the code below to the page and confirm that it displays the data for the correct result.

```python
tdlr_number = '0654479VSF'
```

Confirm that the information is correct. Did it not work out? Go back and edit your selectors, or be a little broader in the parts of the page you sweep up.

## Step 3: Scraping many pages

Using pandas, read in `trucks-subset.csv`.

## Scrape every single row, creating a list of dictionaries from the scraped data.

You should never use for loops with pandas *except when working with Playwright*. To use loops in pandas you'll make use of `.iterrows()`.

The code below loops through a dataframe called `df` and prints out the `address` column.

```python
for index, row in df.iterrows():
    print(row['address'])
```

You'll adapt this code to use your dataframe, and combine it with the scraping code you wrote above.

> ***Tip:** This is like what we did for the townships in class*

## Save your dataframe as `data-uncleaned.csv`

# Cleaning your data

## Re-open the `data-uncleaned.csv` file

You probably want to set `pd.options.display.max_colwidth`

## Clean it up!

Make sure there are columns for

- Business name
- Phone number
- License status
- Physical address

And drop all of the other columns (The easiest way is to use `df = df.drop(columns=[...])`)

## Combine with the original CSV file

To combine dataframes based on indexes, you use `df.join(other_df)`. If you'd prefer to match by columns, you could also use `df.merge(other_df, left_on='...', right_on='...')` and tell it the two columns that match between the two.

### Save to a CSV